In [2]:
%pip install finta
%pip install backtesting
%pip install plotly
%pip install nbformat --upgrade
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
import requests
import json
from finta import TA
import pandas as pd

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
                                              0.0/77.6 kB ? eta -:--:--
     ---------------------------------------- 77.6/77.6 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: nbformat
    Found existing installation: nbformat 5.8.0
    Uninstalling nbformat-5.8.0:
      Successfully uninstalled nbformat-5.8.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Gathering hourly price data for multiple symbols and putting the data in a dictionary.
def create_price_data_dict(symbols, start_date, end_date):
    dict = {}
    
    for symbol in symbols:
        url = f'https://api.pro.coinbase.com/products/{symbol}-USD/candles'
        params = {'start': start_date, 'end': end_date, 'granularity': 3600}
        response = requests.get(url, params=params)
        data = json.loads(response.text)

        price_data_df = pd.DataFrame(data, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
        price_data_df['Date'] = pd.to_datetime(price_data_df['Date'], unit='s')
        price_data_df.set_index('Date', inplace=True)
        price_data_df.sort_index(inplace=True)
        
        dict[symbol] = price_data_df

    return dict

symbols = ['BTC', 'ETH', 'LINK', 'ADA', 'DOGE']
start_date = '2023-04-07T00:00:00.000Z'
end_date = '2023-04-18T23:59:59.999Z'

price_data_dict = create_price_data_dict(symbols, start_date, end_date)
price_data_dict

{'BTC':                          Open      High       Low     Close      Volume
 Date                                                                   
 2023-04-07 00:00:00  28016.06  28120.71  28052.74  28114.90  140.028971
 2023-04-07 01:00:00  28054.16  28117.84  28114.89  28078.18  144.088551
 2023-04-07 02:00:00  27945.38  28095.81  28077.30  28040.70  256.141923
 2023-04-07 03:00:00  28000.00  28070.32  28040.70  28052.55  184.706539
 2023-04-07 04:00:00  28010.19  28085.48  28052.55  28039.49  230.529889
 ...                       ...       ...       ...       ...         ...
 2023-04-18 19:00:00  30107.55  30247.87  30159.51  30224.17  483.065434
 2023-04-18 20:00:00  30156.22  30463.84  30223.35  30426.34  549.306298
 2023-04-18 21:00:00  30281.92  30451.17  30425.49  30378.06  479.996034
 2023-04-18 22:00:00  30298.21  30414.89  30378.06  30378.23  306.597932
 2023-04-18 23:00:00  30308.77  30426.40  30379.18  30393.40  411.325301
 
 [288 rows x 5 columns],
 'ETH':          

In [5]:
# Creating the trading strategy used to analyze over the price data.

class SmaCrossWithRSI(Strategy):
    # Initialize variables
    short_sma = 20
    long_sma = 50
    atr_length = 14
    rsi_length = 14

    # Initialize the strategy. Instances are stored as attributes in our SmaCrossWithRSI object. 
    def init(self):
        # Creates pandas series of the price data for later use. 
        open = pd.Series(self.data.Open)
        close = pd.Series(self.data.Close)
        high = pd.Series(self.data.High)
        low = pd.Series(self.data.Low)
        volume = pd.Series(self.data.Volume)

        # Creates df for the ohlcv data to calculate ATR and RSI.
        ohlcv_df = pd.DataFrame({'open': open, 'high': high, 'low': low, 'close': close, 'volume': volume})

        # Creates instance of ATR and RSI indicators from finta, and the native SMA indicator from bt.
        self.atr = self.I(TA.ATR, ohlcv_df, self.atr_length)
        self.rsi = self.I(TA.RSI, ohlcv_df, self.rsi_length)
        self.sma1 = self.I(SMA, close, self.short_sma)
        self.sma2 = self.I(SMA, close, self.long_sma)

    # Logic for trade entrance and exit.
    def next(self):
        # Price set to the candles 'close' value, used for calculating stop loss and take profit targets.
        price = self.data.Close

        # ATR and RSI for the latest candle.
        ATR = self.atr[-1]
        RSI = self.rsi[-1]

        # If sma1 crosses over sma2, and RSI > 50, then enter a long position with the desired target logic.
        if crossover(self.sma1, self.sma2) and RSI > 50:
            stop_loss = price - (1.1 * ATR)
            take_profit = price + (2 * ATR)
            self.buy(sl=stop_loss, tp=take_profit)

In [6]:
# Running the backtests for each coin.

def run_backtests(price_data_dict, strategy_class):
    dict = {}
    
    for symbol, price_data_df in price_data_dict.items():
        bt = Backtest(price_data_df, strategy_class, cash=100000, commission=0.002, exclusive_orders=True)
        strategy_trade_results = bt.run()
        strategy_trade_results_df = pd.DataFrame(strategy_trade_results).transpose()
        dict[symbol] = strategy_trade_results_df

    return dict

backtest_results_dict = run_backtests(price_data_dict, SmaCrossWithRSI)

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


In [7]:
# Formatting and gathering only the desired metrics to look at for each coin.

desired_metrics = ['# Trades', 'Return [%]', 'Win Rate [%]', 'Sharpe Ratio']
desired_metrics_dict = {}

for symbol, backtest_result_df in backtest_results_dict.items():
    desired_metrics_dict[symbol] = backtest_result_df.loc[0, desired_metrics]

# Combine the desired metrics of the coins into a single DataFrame
desired_metrics_df = pd.DataFrame(desired_metrics_dict).transpose()
desired_metrics_df['Symbols'] = desired_metrics_df.index
desired_metrics_df.set_index('Symbols', inplace=True)

print(desired_metrics_df)

        # Trades Return [%] Win Rate [%] Sharpe Ratio
Symbols                                              
BTC            3   0.006898    33.333333     0.037013
ETH            4   2.361414         75.0      4.77247
LINK           2  -0.442156         50.0          0.0
ADA            2  -1.174481          0.0          0.0
DOGE           2   2.506253        100.0     5.216987


In [8]:
# Aggregating the trading strategy results for all the coins into one dataframe.

total_trades = desired_metrics_df['# Trades'].sum()
avg_return = desired_metrics_df['Return [%]'].mean()
avg_sharpe_ratio = desired_metrics_df['Sharpe Ratio'].mean()
avg_win_rate = desired_metrics_df['Win Rate [%]'].mean()

# Create a DataFrame with the aggregated data
aggregated_df = pd.DataFrame(data=[[total_trades, avg_return, avg_win_rate, avg_sharpe_ratio]], columns=desired_metrics)
aggregated_df.set_index('# Trades', inplace=True)
print(aggregated_df)

          Return [%]  Win Rate [%]  Sharpe Ratio
# Trades                                        
13          0.651586     51.666667      2.005294


In [9]:
# Create subplots: 2 rows, 2 columns
fig = make_subplots(rows=2, cols=2, subplot_titles=("# Trades", "Return [%]", "Win Rate [%]", "Sharpe Ratio"))

# Add traces
fig.add_trace(go.Bar(x=desired_metrics_df.index, y=desired_metrics_df['# Trades'], name='# Trades'), row=1, col=1)
fig.add_trace(go.Bar(x=desired_metrics_df.index, y=desired_metrics_df['Return [%]'], name='Return [%]'), row=1, col=2)
fig.add_trace(go.Bar(x=desired_metrics_df.index, y=desired_metrics_df['Win Rate [%]'], name='Win Rate [%]'), row=2, col=1)
fig.add_trace(go.Bar(x=desired_metrics_df.index, y=desired_metrics_df['Sharpe Ratio'], name='Sharpe Ratio'), row=2, col=2)

# Update xaxis properties
fig.update_xaxes(title_text="Symbols", row=1, col=1)
fig.update_xaxes(title_text="Symbols", row=1, col=2)
fig.update_xaxes(title_text="Symbols", row=2, col=1)
fig.update_xaxes(title_text="Symbols", row=2, col=2)

# Update yaxis properties
fig.update_yaxes(title_text="# Trades", row=1, col=1)
fig.update_yaxes(title_text="Return [%]", row=1, col=2)
fig.update_yaxes(title_text="Win Rate [%]", row=2, col=1)
fig.update_yaxes(title_text="Sharpe Ratio", row=2, col=2)

# Update layout
fig.update_layout(height=700, width=900, title_text="Desired Metrics")

# Show the plot
fig.show()